**Generating a Gaussian Random field** 


CAUTION: SECTION NOT FINISHED\\


A stationary isotropic Gaussian random field $\tilde{Y}(\mathbf{x})$ defined on a 2D domain $\Omega$ is a random function completely specified by its mean and covariance
\begin{align*}
\mu  &= \mathbb{E}[\tilde{Y}(\mathbf{x})], \\
C(||\delta \mathbf{x}||) &= \mathbb{E}[\tilde{Y}(\mathbf{x}) \tilde{Y}(\mathbf{x} + \delta \mathbf{x})] - \mu^2,
\end{align*}
where by stationarity $\mathbf{x}$ can be chosen aribtrarily and by isotropy $C$ depends only on the distance $||\delta \mathbf{x}||$. In addition this function has the following properies:
- The random variable $Y_i = Y(\mathbf{x}_i)$ (corresponding to an evaluation of $Y$ at $X_i$) is normally distributed.
- The vector of random variables $\mathbf{Y} = [Y_1, Y_2, \cdots, Y_n]$ consisting on $n$ point evaluations is joint normal.

Letting $r = ||\delta \mathbf{x}||$ we can relate the covariance $C$ to the power spectral density
\begin{equation}
P(\mathbf{k}) = \int e^{i \langle \mathbf{k} , \delta \mathbf{x} \rangle} C(||\delta \mathbf{x}||) d \delta \mathbf{x},
\end{equation}
using the [Hankel-transform](https://en.wikipedia.org/wiki/Hankel_transform). As number of covariance and power-spectrum pairs can be computed analytically we will use this for verification of our numerical implementation of our Gaussian random field generating function.

To generate $\tilde{Y}(\mathbf{x}) \in \mathbb{R}^d$ we make use of $W(\mathbf{x}) \in \mathbb{C}^d$ a Gaussian White noise random field such that $\mathbb{E}[W(x)W(y)] = \delta(x-y)$. Letting the Fourier transform of this white noise process be defined as
\begin{equation}
FW(k) \equiv F \circ W(x) = \int_{\mathbb{R}^d} e^{2 \pi i (k,x)} W(x) dx,
\end{equation}
we can then write
\begin{equation}
\tilde{Y}(x) = (F^{-1} P^{1/2} F W)(x),
\end{equation}
where $P(k)$ is the power spectrum of the correlation function as defined above. Below we implement two examples using the scale free power law spectrum $P(k) = |k|^{-\alpha}$. As the power spectrum that falls off like a power law and the structures in these fields are typically fractal they do not distinguish any particular length scale.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to generate a Gaussian Random field
def gaussian_random_field(power_spectrum, shape = (128, 128), stat_real=False):
  """
  Generate a scale free Gaussian Random field Y(X) using the
  fast Fourier transform (FFT) and the power spectral density P(k)

  Y(x) = IFFT( P(k) * FFT(W(X)) )(X)

  where W(X) is a Gaussian white noise random field.

  Parameters
  ----------
  power_spectrum : (callable)
    Power spectral density P(k)
  shape : (int, int) G
    Grid size of the field generated

  Returns
  -------
  Y(X) : (array)
    Gaussian random field
  """
  if not callable(power_spectrum):
    raise Exception("`power_spectrum` should be callable")

  def statistic(shape):
    # Build a unit-distribution of complex numbers with random phase
    a = np.random.normal(loc=0, scale=1, size=shape)
    b = np.random.normal(loc=0, scale=1, size=shape)
    return a + 1j * b

  # Compute the k grid
  fftfreq = np.fft.fftfreq
  all_k = [fftfreq(s, d=1/s) for s in shape]
  kgrid = np.meshgrid(*all_k, indexing="ij")  # Reutrns kX, kY
  knorm = np.sqrt(np.sum(np.power(kgrid, 2), axis=0))  # Computes ||k|| = \sqrt{kX^2 + kY^2}

  # (1) Compute the Fourier transform of Gaussian White noise W(X)
  if stat_real:
    W = statistic(shape)
    # Compute the FFT of the field
    FW = np.fft.fftn(W)
  else:
    # Draw a random sample in Fourier space
    fourier_shape = knorm.shape
    FW = statistic(fourier_shape)

  # (2) Multiply by P(k)^{1/2}
  power_k = np.zeros_like(knorm)
  mask = knorm > 0
  power_k[mask] = np.sqrt(power_spectrum(knorm[mask]))
  FW *= power_k

  # (3) Take its IFFT
  return  np.real(np.fft.ifftn(FW))

Below we create an example Gaussian random field.

In [ ]:
# Create an example
n_pts = 512
power_spectrum = lambda k: k**-3
shape = (n_pts, n_pts)

Z_1 = np.linspace(-np.pi,np.pi,num=n_pts)
Z_2 = np.linspace(-np.pi,np.pi,num=n_pts)
Ys = gaussian_random_field(power_spectrum, shape) # Ys for Y surrogate

# plot Ys(Z) as a field
plt.figure(figsize=(4,4))
cmap = plt.get_cmap('RdBu')
plt.pcolormesh(Z_1, Z_2, Ys, cmap=cmap)
plt.xlabel(r'$Z_1$', fontsize=20)
plt.ylabel(r'$Z_2$', fontsize=20)
plt.xticks([])
plt.yticks([])
plt.show()

Next we validate our implementation. To do so we specify the correlation function
\begin{equation}
C(||\delta \mathbf{x}||) = r, %e^{\frac{-1}{2} r^2},
\end{equation}
for which the power spectrum is given by
\begin{equation}
P(\mathbf{k}) = \int e^{i \langle \mathbf{k} , \delta \mathbf{x} \rangle} C(||\delta \mathbf{x}||) d \delta \mathbf{x} = -\frac{1}{k^3}.
\end{equation}
We then generate a field with this power spectrum and compare its computed covariance with the analytical expression.

In [ ]:
# 1) Generate a field
n_pts = 256
power_spectrum = lambda k: 1/k**3#np.exp(-.5*k**2)
shape = (n_pts, n_pts)
Ys = gaussian_random_field(power_spectrum, shape) # Ys for Y surrogate
plt.imshow(Ys)
plt.show()

# 2) Compute the covariance
from scipy.signal import correlate
C = correlate(Ys,Ys,mode='same',method='auto')
plt.imshow(C)
plt.show()

# 3) Compute the error

indx = np.zeros(n_pts)
indx[0:n_pts//2] = np.arange(-n_pts//2,0,1)
indx[n_pts//2:] = np.arange(1,n_pts//2+1,1)

# create array of radii
x,y = np.meshgrid(indx,indx)
R = np.sqrt(x**2 + y**2)

C_anal = 1/R

# calculate the mean
r  = np.linspace(1,n_pts//2,num=n_pts//2)
f = lambda r : C[(R >= r-.5) & (R < r+.5)].mean()
mean_num = np.vectorize(f)(r)
f = lambda r : C_anal[(R >= r-.5) & (R < r+.5)].mean()
mean_anal = np.vectorize(f)(r)
# plot it
fig,ax=plt.subplots()
ax.plot(r,mean_num/mean_num[0],'b-')
ax.plot(r,mean_anal,'k') # Analytical
ax.plot(r,1/r,'k:') # Analytical

ax.set_xlabel(r'$r$')
ax.set_ylabel(r'$C(r)$')
plt.show()

In addition to the brief overview given here some nice texts on this topic are [1](https://structures.uni-heidelberg.de/blog/posts/gaussian-random-fields/index.php) and [2](https://arxiv.org/pdf/1105.2737).